# Solver を利用して線形最適化問題を解いてみよう
主婦の問題を簡単にしたものを解いていく。
３種類の食料品と４種類の栄養素の組み合わせを考えていく。

In [1]:
import numpy as np
import pandas as pd
from itertools import product
from pulp import LpVariable, lpSum, value
from ortoolpy import model_min, addvars, addvals
from IPython.display import display

In [7]:
# 主婦の問題を Solver によって解く

# Data の読み込み
df_n = pd.read_csv('nutrition.csv', index_col='食料品')
df_p = pd.read_csv('price.csv')
print("食料品と栄養素の関係")
display(df_n)
print("食料品の価格")
display(df_p)

食料品と栄養素の関係


,栄養素１,栄養素２,栄養素３,栄養素４
食料品,,,,
食料品１,0,0,90,10
食料品２,10,50,40,70
食料品３,80,0,0,0


食料品の価格


,食料品１,食料品２,食料品３
0,1000,150,300


In [8]:
# 初期設定
np.random.seed(1)
n_p = len(df_n.index)
nn = len(df_n.columns)
pr = list(range(n_p))

In [9]:
# 数理Model 作成
m1 = model_min()
# 目的関数
v1 = {(i):LpVariable('v%d'%(i), cat='Integer', lowBound=0) for i in pr}
# 制約条件
m1 += lpSum(df_p.iloc[0][i] * v1[i] for i in pr)
for j in range(nn):
    m1 += lpSum(v1[i] * df_n.iloc[i][j] for i in range(n_p)) >= 100
m1.solve()

1

In [10]:
# 総Cost 計算
print("最適解")
total_cost = 0
for k, x in v1.items():
    i = k
    print(f"{df_n.index[i]}の個数: {int(value(x))}個")
    total_cost += df_p.iloc[0][i] * value(x)

print(f"総Cost: {int(total_cost)}円")

最適解
食料品１の個数: 0個
食料品２の個数: 3個
食料品３の個数: 1個
総Cost: 750円


## 5-3. 非線形最適化問題を解いてみよう
正方形の形をした土地の最適な長さを求める問題を考える。
- 目的: x の二乗と 1000 との差を最小化する
- 条件: x は正の数

In [11]:
# 二分探索によって f(x) が 0 となる x の値を求める

def f(x):
    return x ** 2 - 1000

In [13]:
# 初期設定
lo = -0.1
hi = 1000.1
eps = 1e-10  # 許容誤差

In [14]:
# 二分探索を実行
count = 0
while hi - lo > eps:
    x = (lo + hi) / 2
    if f(x) >= 0:
        hi = x
    else:
        lo = x
    count += 1

print(f"結果: {hi}")
print(f"探索回数: {count}回")

結果: 31.622776601731587
探索回数: 44回


二分探索：両端から範囲を２つに分けながら、少しずつ絞り込みを行なっていく方法

In [15]:
# Newton法によって f(x) が 0 となる x の値を求める

# Newton法の関数
# x0, eps は Default値
def square_root(y, x0=1, eps=1e-10):
    x = x0
    count = 0
    while abs(x ** 2 - y) > eps:
        x -= (x * x - y) / (2 * x)
        count += 1
    return x, count


# Newton法の実行
x, count = square_root(1000)
print(f"結果: {x}")
print(f"探索回数: {count}回")

結果: 31.622776601684333
探索回数: 9回


Newton法:
関数と x軸との交点を求める手法。関数の値が 0 になる x の値を求める方法として用いられる。

Newton法の法が二分探索に比べて高速に計算ができた。